In [14]:
import pickle
from extract_consecutive_screenshots import get_pickle_name, load_objects, load_snapshot_data, analyze_run
from visualization import get_connection, get_gta_image_jpg_dir
from os import path
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import patches
import numpy as np
import progressbar
import os
# JPEG_DIR = get_gta_image_jpg_dir()
JPEG_DIR = 'D:/GTAV_extraction_output/rgb-jpeg'

In [15]:
run_id = 52
analyze_run(run_id)

In [16]:
cars = load_objects(run_id)
len(cars)

6782

In [17]:
def get_pos_diffs(snapshots):
    pos_diffs = []
    positions = [snap['position'] for snap in snaps]
    for i in range(1, len(snaps)):
        diff = np.array(positions[i]) - np.array(positions[i - 1])
        pos_diffs.append(diff)
    return pos_diffs

def get_time_diffs(snapshots):
    # gets difference in seconds, for normalization
    time_diffs = []
    times = [snap['timestamp'] for snap in snapshots]
    for i in range(1, len(snapshots)):
        diff = (times[i] - times[i - 1]).total_seconds()
        time_diffs.append(diff)
    return time_diffs

def transform_to_relative(diff, cam):
    base = np.array([
        [cam[0], cam[1]],
        [cam[1], -cam[0]]
    ])
    return np.matmul(base, diff[0:2])

def normalize_diff(relative_diff, time_diff):
    return relative_diff / time_diff
    
def get_pickle_processed_name(run_id):
    return os.path.join('runs', 'run_{}_processed_pickle.rick'.format(run_id))

def save_processed_objects(run_id, objects):
    data_file = get_pickle_processed_name(run_id)
    with open(data_file, 'wb+') as file:
        pickle.dump(objects, file)


In [18]:
widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ', progressbar.FileTransferSpeed()]
pbar = progressbar.ProgressBar(widgets=widgets, max_value = len(cars)).start()
counter = 0

for i, tupl in enumerate(cars.items()):
    key, car = tupl
    pbar.update(i)
    snaps = car['snapshots']
    pos_diffs = get_pos_diffs(snaps)
    time_diffs = get_time_diffs(snaps)
    for i in range(len(pos_diffs)):
        snap = snaps[i]
        diff = pos_diffs[i]
        time_diff = time_diffs[i]
        diff_rel = transform_to_relative(diff, snap['cam_direction'][0:2])
        diff_normalized = normalize_diff(diff_rel, time_diff)
        
        snap['diff'] = diff
        snap['diff_relative'] = diff_rel
        snap['diff_normalized'] = diff_normalized
        snaps[i]
pbar.finish()

save_processed_objects(run_id, cars)

100% 6782 |#######################################################|   2.0 KiB/s
